In [10]:
import pygame
import sys

import math
from itertools import product
import time
fov=85

prev=[]

frame_rate = 2
def size(d, a):
    #assuming it is a list of objects, given in the parameter
    width = 0
    length = 0
    height = 0
    angs, dim=[],[]
    for i, j in zip([0,0,1], [1,2,2]):
        ang=math.radians(abs(a[i]-a[j]))
        d1, d2=d[i],d[j]
        temp = ((d1**2) + (d2**2) - (2*d1*d2*math.cos(ang)))**(1/2)
        dim.append(temp)
        angs.append(ang)
       
    d1,d2,d3=dim[0],dim[1], dim[2]
    if 0<=abs(d1-d2)<=1:
        base=d3/2
        slant=d1

    elif 0<=abs(d3-d2)<=1:
        base=d1/2
        slant=d3
    else:
        base=d2/2
        slant=d1
    height=(abs((slant**2)-(base**2)))**0.5
    area= round(base*height,2)
    return area

def findspeed(p, d):
    return round(sum([abs(a-b) for a,b in zip(p,d)])/3,1)


    
pygame.init()

# Set up the display
width, height = 1200, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption('Simulator')

# Set up the initial red sphere properties
initial_radius = 25
radius = initial_radius
x, y = 300+width // 2, 100
points = [(x, y), (x + 10, y - 50), (x - 10, y - 50)]
speed = 10 # Smaller step size
ori = 180
ang = 5
font = pygame.font.Font(None, 20)  # Choose a font and size for the text
start = 0
clock = pygame.time.Clock()
data = []
positions = []
running = True
# Colors
blue = (41,144 , 246)
green=(0, 255, 0)
red = (255, 0, 0)

# Load images
cargo_image = pygame.image.load("cargo.png")  # Replace "cargo.png" with your cargo image file
boat_images = [
    pygame.image.load("boat.png")
    # Add more boat images as needed
]
# Initial positions

# Scaled image dimensions
scaled_cargo_width = int(cargo_image.get_width() * 0.45)
scaled_cargo_height = int(cargo_image.get_height() * 0.75)
scaled_boat_width = int(boat_images[0].get_width() * 0.3)
scaled_boat_height = int(boat_images[0].get_height() * 0.36)

cargo_pos = [width // 4 - scaled_cargo_width // 2, height - scaled_cargo_height]

# Rectangle dimensions for collision detection (customize as needed)
cargo_collision_rect = pygame.Rect(cargo_pos[0]-10, cargo_pos[1]-10, scaled_cargo_width+20, scaled_cargo_height+20)

# Rectangle dimensions for image drawing (customize as needed)
cargo_image_rect = pygame.Rect(cargo_pos[0]-10, cargo_pos[1]-10, scaled_cargo_width+20, scaled_cargo_height+20)
collision = False
speed1=0
ori2=0
while running:
    dflag = True
    flag = True
    sflag=False
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            sflag=True
            mx, my = pygame.mouse.get_pos()
            p2 = []
            cx = sum(point[0] for point in points) // 3
            cy = sum(point[1] for point in points) // 3
            #if len(positions) == 0:
             #   positions.append((int(cx), int(cy)))
            #elif positions[-1][0] != int(cx) and positions[-1][1] != int(cy):
            positions.append((int(cx-600), int(cy)))
            c = 0
            for point in points:
                x, y = point
                if width - 150 <= mx <= width - 110 and height - 550 <= my <= height - 530:  # top
                    new_y = y - speed * (math.cos(math.radians(ori)))
                    new_x = x + speed * (math.sin(math.radians(ori)))
                elif width - 170 <= mx < width - 150 and height - 520 <= my <= height - 480:  # left
                    x -= cx
                    y -= cy
                    new_x = x * math.cos(math.radians(ang)) - y * math.sin(math.radians(ang))
                    new_y = x * math.sin(math.radians(ang)) + y * math.cos(math.radians(ang))
                    new_x += cx
                    new_y += cy
                    if c == 0:
                        ori = abs((ori + ang) % 360)
                        c = 1

                elif width - 110 <= mx <= width - 90 and height - 520 <= my <= height - 480:  # right
                    x -= cx
                    y -= cy
                    new_x = x * math.cos(math.radians(-ang)) - y * math.sin(math.radians(-ang))
                    new_y = x * math.sin(math.radians(-ang)) + y * math.cos(math.radians(-ang))
                    new_x += cx
                    new_y += cy
                    if c == 0:
                        ori = abs((ori - ang) % 360)
                        c = 1

                elif width - 150 <= mx < width - 110 and height - 470 <= my <= height - 450:  # bottom
                    new_y = y + speed * (math.cos(math.radians(ori)))
                    new_x = x - speed * (math.sin(math.radians(ori)))
                else:
                    new_x, new_y = x, y

                if new_x != max(600+radius, min(new_x, width - radius)) or new_y != max(radius, min(new_y, height - radius)):
                    flag = False
                    ori=ori2
                    break
                p2.append((int(new_x), int(new_y)))
                


            if flag:
                points = p2.copy()
                ori2=ori

    # Ensure the sphere stays within the screen boundaries
    # x = max(radius, min(x, width - radius))
    # y = max(radius, min(y, height - radius))
    # x_l, y_l = max(radius, min(x_l, width - radius)), max(radius, min(y_l, height  - radius))
    # x_r, y_r = max(radius, min(x_r, width - radius)), max(radius, min(y_r, height  - radius))

    # Clear the screen
    pygame.draw.rect(screen, (0,0,255), (0, 0, width // 2, height))
    pygame.draw.rect(screen, (0,0,0), (width // 2, 0, width // 2, height))
    diameters = []
    angles = []
    # Draw the red sphere
    for point in points:
        x, y = point
        r = int(1+y/30)
        angle = round((start + ((x-600) * fov*2) / 600) % 360, 1)
        diameter = 2 * r
        angles.append(angle)
        diameters.append(diameter)
        font = pygame.font.Font(None, 20)
        pygame.draw.circle(screen, (138, 18, 26), (int(x), int(y)), r)
        if points[1] == point:
            screen.blit(font.render(f'{2 * r}', True, (83, 235, 46)), (x - 25, y - 20))
            screen.blit(font.render(f'{round(angle, 1)}', True, (223, 239, 119)), (x, y - 20))
        else:
            screen.blit(font.render(f'{2 * r}', True, (83, 235, 46)), (x - 40, y - 20))
            screen.blit(font.render(f'{round(angle, 1)}', True, (223, 239, 119)), (x-15, y - 20))

    # Draw arrows
    pygame.draw.polygon(screen, (0, 0, 255), [(width - 150, height - 530), (width - 110, height - 530),
                                               (width - 130, height - 550)])
    pygame.draw.polygon(screen, (0, 0, 255), [(width - 150, height - 520), (width - 150, height - 480),
                                               (width - 170, height - 500)])
    pygame.draw.polygon(screen, (0, 0, 255), [(width - 110, height - 520), (width - 110, height - 480),
                                               (width - 90, height - 500)])
    pygame.draw.polygon(screen, (0, 0, 255), [(width - 150, height - 470), (width - 110, height - 470),
                                               (width - 130, height - 450)])

    # Display the diameter text
    diameter_text = font.render(f'System Input: Camera vision at night', True, (255, 255, 255))
    screen.blit(diameter_text, (610, 10))
    diameter_text = font.render(f'Diameter in pixels', True, (83, 235, 46))
    screen.blit(diameter_text, (610, 30))
    diameter_text = font.render(f'Angle in degrees', True, (223, 239, 119))
    screen.blit(diameter_text, (610, 50))
    
    ori1=ori+90
    k= 1500*0.8/(2*math.tan(math.radians(fov)))
    distances= [k/(x) for x in diameters]
    area= size(distances, angles)
    if len(prev)>0: 
        if prev[0]!=distances[0]:
            speed1=(distances[0]-prev[0])*frame_rate
    prev=distances.copy()
    distance=min(distances)#round(sum(distances)/3,2)
    #if len(positions)<2:
     #   position=(500, 300)
    #else:
     #   position=positions[-1]
    d= sum(distances)/3
    a=math.radians(sum(angles)/3)
    position= 0.5*pygame.math.Vector2(int(340+cargo_image_rect.centerx+d*math.sin(a)),int(340+cargo_image_rect.centery-d*math.cos(a)))
    
    boat_pos =position#pygame.math.Vector2(300,150)+0.2*((pygame.math.Vector2(600-position[0], abs(600-position[1]))-pygame.math.Vector2(positions[-2])).rotate(-90)+pygame.math.Vector2(positions[-2])) if len(positions)>=2 else (position[0], abs(position[1]))
    boat_collision_rect = pygame.Rect(boat_pos[0], boat_pos[1], scaled_boat_width, scaled_boat_height)
    boat_image_rect = pygame.Rect(boat_pos[0], boat_pos[1], scaled_boat_width, scaled_boat_height)
    ###
    rotated_boat = pygame.transform.rotate(pygame.transform.scale(boat_images[0], (scaled_boat_width, scaled_boat_height)), -ori1)

    # Update rectangles for collision detection
    boat_collision_rect.update(boat_pos[0], boat_pos[1], rotated_boat.get_width(), rotated_boat.get_height())
    # Update rectangles for image drawing
    
    boat_image_rect.update(boat_pos[0], boat_pos[1], rotated_boat.get_width(), rotated_boat.get_height())

    # Check for collision between rectangles
    if cargo_collision_rect.colliderect(boat_collision_rect):
        collision = True
    else:
        collision = False
        
    pygame.draw.polygon(screen, (65,89,255), [(cargo_image_rect.centerx,cargo_image_rect.centery),(int(cargo_image_rect.centerx+((600-cargo_image_rect.centery)*math.tan(math.radians(10)))),600), (600,600),(600,0),(cargo_image_rect.centerx, 0)])
     # Draw vessels (images)
    rotated_cargo = pygame.transform.rotate(pygame.transform.scale(cargo_image, (scaled_cargo_width, scaled_cargo_height)), 0)
    screen.blit(rotated_cargo, cargo_pos)
    screen.blit(rotated_boat, boat_pos)

    # Draw rectangles around vessels (for image drawing)
    pygame.draw.rect(screen, red if collision else green, cargo_image_rect, 2)
    pygame.draw.rect(screen, red if collision else green, boat_image_rect, 2)

    #pygame.draw.line(screen, [12,28,56], (cargo_image_rect.centerx,cargo_image_rect.centery), (cargo_image_rect.centerx,0), width=1)
    #pygame.draw.line(screen, [12,28,56], (cargo_image_rect.centerx,cargo_image_rect.centery), (int(cargo_image_rect.centerx+((600-cargo_image_rect.centery)*math.tan(math.radians(10)))),600), width=1)
    
    diameter_text = font.render(f'System Output', True, (255, 255, 255))
    screen.blit(diameter_text, (10, 10))
    diameter_text = font.render(f'HFOV: {start}-{start+fov*2}deg', True, (255, 255, 255))
    screen.blit(diameter_text, (310, 10))
    diameter_text = font.render(f'Horizontal resolution:1500px', True, (255, 255, 255))
    screen.blit(diameter_text, (310, 30))
    diameter_text = font.render(f'Actual diameter: 80cm', True, (255, 255, 255))
    screen.blit(diameter_text, (10, 50))
    diameter_text = font.render(f'Activates after two inputs', True, (255, 255, 255))
    screen.blit(diameter_text, (10, 30))
    
    ###
    screen.blit(font.render(f'Type:Fising boat, Area:{area}sqm', True, (0,0,0)), (boat_pos[0], boat_pos[1]-60))
    screen.blit(font.render(f'Distance:{round(distance,1)}m', True, (0,0,0)),(boat_pos[0], boat_pos[1]-40))
    screen.blit(font.render(f'Velocity:{round(speed1,1)}m/s', True, (0,0,0)),(boat_pos[0], boat_pos[1]-20))
    
    
    # Update the display
    pygame.display.flip()

pygame.quit()
sys.exit()


SystemExit: 

In [3]:
import pygame
import sys

import math
from itertools import product
import time
fov=60

prev=[]

frame_rate = 2
def size(d, a):
    #assuming it is a list of objects, given in the parameter
    width = 0
    length = 0
    height = 0
    angs, dim=[],[]
    for i, j in zip([0,0,1], [1,2,2]):
        ang=math.radians(abs(a[i]-a[j]))
        d1, d2=d[i],d[j]
        temp = ((d1**2) + (d2**2) - (2*d1*d2*math.cos(ang)))**(1/2)
        dim.append(temp)
        angs.append(ang)
       
    d1,d2,d3=dim[0],dim[1], dim[2]
    if 0<=abs(d1-d2)<=1:
        base=d3/2
        slant=d1

    elif 0<=abs(d3-d2)<=1:
        base=d1/2
        slant=d3
    else:
        base=d2/2
        slant=d1
    height=(abs((slant**2)-(base**2)))**0.5
    area= round(base*height,2)
    return area

def findspeed(p, d):
    return round(sum([abs(a-b) for a,b in zip(p,d)])/3,1)


    
pygame.init()

# Set up the display
width, height = 1200, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption('Simulator')

# Set up the initial red sphere properties
initial_radius = 25
radius = initial_radius
x, y = 300+width // 2, 100
points = [(x, y), (x + 10, y - 50), (x - 10, y - 50)]
speed = 10 # Smaller step size
ori = 180
ang = 5
font = pygame.font.Font(None, 20)  # Choose a font and size for the text
start = 0
clock = pygame.time.Clock()
data = []
positions = []
running = True
# Colors
blue = (41,144 , 246)
green=(0, 255, 0)
red = (255, 0, 0)

# Load images
cargo_image = pygame.image.load("cargo.png")  # Replace "cargo.png" with your cargo image file
boat_images = [
    pygame.image.load("boat.png")
    # Add more boat images as needed
]
# Initial positions

# Scaled image dimensions
scaled_cargo_width = int(cargo_image.get_width() * 0.45)
scaled_cargo_height = int(cargo_image.get_height() * 0.75)
scaled_boat_width = int(boat_images[0].get_width() * 0.3)
scaled_boat_height = int(boat_images[0].get_height() * 0.36)

cargo_pos = [width // 4 - scaled_cargo_width // 2, height - scaled_cargo_height]

# Rectangle dimensions for collision detection (customize as needed)
cargo_collision_rect = pygame.Rect(cargo_pos[0]-10, cargo_pos[1]-10, scaled_cargo_width+20, scaled_cargo_height+20)

# Rectangle dimensions for image drawing (customize as needed)
cargo_image_rect = pygame.Rect(cargo_pos[0]-10, cargo_pos[1]-10, scaled_cargo_width+20, scaled_cargo_height+20)
collision = False
speed1=0
ori2=0
while running:
    dflag = True
    flag = True
    sflag=False
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            sflag=True
            mx, my = pygame.mouse.get_pos()
            p2 = []
            cx = sum(point[0] for point in points) // 3
            cy = sum(point[1] for point in points) // 3
            #if len(positions) == 0:
             #   positions.append((int(cx), int(cy)))
            #elif positions[-1][0] != int(cx) and positions[-1][1] != int(cy):
            positions.append((int(cx-600), int(cy)))
            c = 0
            for point in points:
                x, y = point
                if width - 150 <= mx <= width - 110 and height - 550 <= my <= height - 530:  # top
                    new_y = y - speed * (math.cos(math.radians(ori)))
                    new_x = x + speed * (math.sin(math.radians(ori)))
                elif width - 170 <= mx < width - 150 and height - 520 <= my <= height - 480:  # left
                    x -= cx
                    y -= cy
                    new_x = x * math.cos(math.radians(ang)) - y * math.sin(math.radians(ang))
                    new_y = x * math.sin(math.radians(ang)) + y * math.cos(math.radians(ang))
                    new_x += cx
                    new_y += cy
                    if c == 0:
                        ori = abs((ori + ang) % 360)
                        c = 1

                elif width - 110 <= mx <= width - 90 and height - 520 <= my <= height - 480:  # right
                    x -= cx
                    y -= cy
                    new_x = x * math.cos(math.radians(-ang)) - y * math.sin(math.radians(-ang))
                    new_y = x * math.sin(math.radians(-ang)) + y * math.cos(math.radians(-ang))
                    new_x += cx
                    new_y += cy
                    if c == 0:
                        ori = abs((ori - ang) % 360)
                        c = 1

                elif width - 150 <= mx < width - 110 and height - 470 <= my <= height - 450:  # bottom
                    new_y = y + speed * (math.cos(math.radians(ori)))
                    new_x = x - speed * (math.sin(math.radians(ori)))
                else:
                    new_x, new_y = x, y

                if new_x != max(600+radius, min(new_x, width - radius)) or new_y != max(radius, min(new_y, height - radius)):
                    flag = False
                    ori=ori2
                    break
                p2.append((int(new_x), int(new_y)))
                


            if flag:
                points = p2.copy()
                ori2=ori

    # Ensure the sphere stays within the screen boundaries
    # x = max(radius, min(x, width - radius))
    # y = max(radius, min(y, height - radius))
    # x_l, y_l = max(radius, min(x_l, width - radius)), max(radius, min(y_l, height  - radius))
    # x_r, y_r = max(radius, min(x_r, width - radius)), max(radius, min(y_r, height  - radius))

    # Clear the screen
    pygame.draw.rect(screen, (0,0,255), (0, 0, width // 2, height))
    pygame.draw.rect(screen, (0,0,0), (width // 2, 0, width // 2, height))
    diameters = []
    angles = []
    # Draw the red sphere
    for point in points:
        x, y = point
        r = int(1+y/30)
        angle = round((start + ((x-600) * fov*2) / 600) % 360, 1)
        diameter = 2 * r
        angles.append(angle)
        diameters.append(diameter)
        font = pygame.font.Font(None, 20)
        pygame.draw.circle(screen, (138, 18, 26), (int(x), int(y)), r)
        if points[1] == point:
            screen.blit(font.render(f'{2 * r}', True, (83, 235, 46)), (x - 25, y - 20))
            screen.blit(font.render(f'{round(angle, 1)}', True, (223, 239, 119)), (x, y - 20))
        else:
            screen.blit(font.render(f'{2 * r}', True, (83, 235, 46)), (x - 40, y - 20))
            screen.blit(font.render(f'{round(angle, 1)}', True, (223, 239, 119)), (x-15, y - 20))

    # Draw arrows
    pygame.draw.polygon(screen, (0, 0, 255), [(width - 150, height - 530), (width - 110, height - 530),
                                               (width - 130, height - 550)])
    pygame.draw.polygon(screen, (0, 0, 255), [(width - 150, height - 520), (width - 150, height - 480),
                                               (width - 170, height - 500)])
    pygame.draw.polygon(screen, (0, 0, 255), [(width - 110, height - 520), (width - 110, height - 480),
                                               (width - 90, height - 500)])
    pygame.draw.polygon(screen, (0, 0, 255), [(width - 150, height - 470), (width - 110, height - 470),
                                               (width - 130, height - 450)])

    # Display the diameter text
    diameter_text = font.render(f'System Input: Camera vision at night', True, (255, 255, 255))
    screen.blit(diameter_text, (610, 10))
    diameter_text = font.render(f'Diameter in pixels', True, (83, 235, 46))
    screen.blit(diameter_text, (610, 30))
    diameter_text = font.render(f'Angle in degrees', True, (223, 239, 119))
    screen.blit(diameter_text, (610, 50))
    
    ori1=ori+90
    k= 1500*0.8/(2*math.tan(math.radians(fov)))
    distances= [k/(x) for x in diameters]
    area= size(distances, angles)
    if len(prev)>0: 
        if prev[0]!=distances[0]:
            speed1=(distances[0]-prev[0])*frame_rate
    prev=distances.copy()
    distance=min(distances)#round(sum(distances)/3,2)
    #if len(positions)<2:
     #   position=(500, 300)
    #else:
     #   position=positions[-1]
    d= sum(distances)/3
    a=math.radians(sum(angles)/3)
    position= 0.5*pygame.math.Vector2(int(340+cargo_image_rect.centerx+d*math.sin(a)),int(340+cargo_image_rect.centery-d*math.cos(a)))
    
    boat_pos =position#pygame.math.Vector2(300,150)+0.2*((pygame.math.Vector2(600-position[0], abs(600-position[1]))-pygame.math.Vector2(positions[-2])).rotate(-90)+pygame.math.Vector2(positions[-2])) if len(positions)>=2 else (position[0], abs(position[1]))
    boat_collision_rect = pygame.Rect(boat_pos[0], boat_pos[1], scaled_boat_width, scaled_boat_height)
    boat_image_rect = pygame.Rect(boat_pos[0], boat_pos[1], scaled_boat_width, scaled_boat_height)
    ###
    rotated_boat = pygame.transform.rotate(pygame.transform.scale(boat_images[0], (scaled_boat_width, scaled_boat_height)), -ori1)

    # Update rectangles for collision detection
    boat_collision_rect.update(boat_pos[0], boat_pos[1], rotated_boat.get_width(), rotated_boat.get_height())
    # Update rectangles for image drawing
    
    boat_image_rect.update(boat_pos[0], boat_pos[1], rotated_boat.get_width(), rotated_boat.get_height())

    # Check for collision between rectangles
    if cargo_collision_rect.colliderect(boat_collision_rect):
        collision = True
    else:
        collision = False
        
    pygame.draw.polygon(screen, (65,89,255), [(cargo_image_rect.centerx,cargo_image_rect.centery),(600,int(cargo_image_rect.centery+((600-cargo_image_rect.centerx)*math.tan(math.radians(30))))), (600,600),(600,0),(cargo_image_rect.centerx, 0)])
     # Draw vessels (images)
    rotated_cargo = pygame.transform.rotate(pygame.transform.scale(cargo_image, (scaled_cargo_width, scaled_cargo_height)), 0)
    screen.blit(rotated_cargo, cargo_pos)
    screen.blit(rotated_boat, boat_pos)

    # Draw rectangles around vessels (for image drawing)
    pygame.draw.rect(screen, red if collision else green, cargo_image_rect, 2)
    pygame.draw.rect(screen, red if collision else green, boat_image_rect, 2)

    #pygame.draw.line(screen, [12,28,56], (cargo_image_rect.centerx,cargo_image_rect.centery), (cargo_image_rect.centerx,0), width=1)
    #pygame.draw.line(screen, [12,28,56], (cargo_image_rect.centerx,cargo_image_rect.centery), (int(cargo_image_rect.centerx+((600-cargo_image_rect.centery)*math.tan(math.radians(10)))),600), width=1)
    
    diameter_text = font.render(f'System Output', True, (255, 255, 255))
    screen.blit(diameter_text, (10, 10))
    diameter_text = font.render(f'HFOV: {start}-{start+fov*2}deg', True, (255, 255, 255))
    screen.blit(diameter_text, (310, 10))
    diameter_text = font.render(f'Horizontal resolution:1500px', True, (255, 255, 255))
    screen.blit(diameter_text, (310, 30))
    diameter_text = font.render(f'Actual diameter: 80cm', True, (255, 255, 255))
    screen.blit(diameter_text, (10, 50))
    diameter_text = font.render(f'Activates after two inputs', True, (255, 255, 255))
    screen.blit(diameter_text, (10, 30))
    
    ###
    screen.blit(font.render(f'Type:Boat, Area:{area}sqm', True, (0,0,0)), (boat_pos[0], boat_pos[1]-60))
    screen.blit(font.render(f'Distance:{round(distance,1)}m', True, (0,0,0)),(boat_pos[0], boat_pos[1]-40))
    screen.blit(font.render(f'Velocity:{round(speed1,1)}m/s', True, (0,0,0)),(boat_pos[0], boat_pos[1]-20))
    
    
    # Update the display
    pygame.display.flip()

pygame.quit()
sys.exit()


SystemExit: 

In [2]:
import pygame
import sys

import math
from itertools import product
import time
fov=30

prev=[]

frame_rate = 2
def size(d, a):
    #assuming it is a list of objects, given in the parameter
    width = 0
    length = 0
    height = 0
    angs, dim=[],[]
    for i, j in zip([0,0,1], [1,2,2]):
        ang=math.radians(abs(a[i]-a[j]))
        d1, d2=d[i],d[j]
        temp = ((d1**2) + (d2**2) - (2*d1*d2*math.cos(ang)))**(1/2)
        dim.append(temp)
        angs.append(ang)
       
    d1,d2,d3=dim[0],dim[1], dim[2]
    if 0<=abs(d1-d2)<=1:
        base=d3/2
        slant=d1

    elif 0<=abs(d3-d2)<=1:
        base=d1/2
        slant=d3
    else:
        base=d2/2
        slant=d1
    height=(abs((slant**2)-(base**2)))**0.5
    area= round(base*height,2)
    return area

def findspeed(p, d):
    return round(sum([abs(a-b) for a,b in zip(p,d)])/3,1)


    
pygame.init()

# Set up the display
width, height = 1200, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption('Simulator')

# Set up the initial red sphere properties
initial_radius = 25
radius = initial_radius
x, y = 300+width // 2, 100
points = [(x, y), (x + 10, y - 50), (x - 10, y - 50)]
speed = 10 # Smaller step size
ori = 180
ang = 5
font = pygame.font.Font(None, 30) # Choose a font and size for the text
start = 0
clock = pygame.time.Clock()
data = []
positions = []
running = True
# Colors
blue = (41,144 , 246)
green=(0, 255, 0)
red = (255, 0, 0)

# Load images
cargo_image = pygame.image.load("cargo.png")  # Replace "cargo.png" with your cargo image file
boat_images = [
    pygame.image.load("boat.png")
    # Add more boat images as needed
]
# Initial positions

# Scaled image dimensions
scaled_cargo_width = int(cargo_image.get_width() * 0.45)
scaled_cargo_height = int(cargo_image.get_height() * 0.75)
scaled_boat_width = int(boat_images[0].get_width() * 0.3)
scaled_boat_height = int(boat_images[0].get_height() * 0.36)

cargo_pos = [width // 4 - scaled_cargo_width // 2, height - scaled_cargo_height]

# Rectangle dimensions for collision detection (customize as needed)
cargo_collision_rect = pygame.Rect(cargo_pos[0]-10, cargo_pos[1]-10, scaled_cargo_width+20, scaled_cargo_height+20)
ptime=time.time()
# Rectangle dimensions for image drawing (customize as needed)
cargo_image_rect = pygame.Rect(cargo_pos[0]-10, cargo_pos[1]-10, scaled_cargo_width+20, scaled_cargo_height+20)
collision = False
speed1=0
ori2=0
e=0
start_time=0
while running:
    dflag = True
    flag = True
    sflag=False
    for event in pygame.event.get():
        
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            sflag=True
            start_time=time.time()
            mx, my = pygame.mouse.get_pos()
            p2 = []
            cx = sum(point[0] for point in points) // 3
            cy = sum(point[1] for point in points) // 3
            #if len(positions) == 0:
             #   positions.append((int(cx), int(cy)))
            #elif positions[-1][0] != int(cx) and positions[-1][1] != int(cy):
            positions.append((int(cx-600), int(cy)))
            c = 0
            for point in points:
                x, y = point
                if width - 150 <= mx <= width - 110 and height - 550 <= my <= height - 530:  # top
                    new_y = y - speed * (math.cos(math.radians(ori)))
                    new_x = x + speed * (math.sin(math.radians(ori)))
                elif width - 170 <= mx < width - 150 and height - 520 <= my <= height - 480:  # left
                    x -= cx
                    y -= cy
                    new_x = x * math.cos(math.radians(ang)) - y * math.sin(math.radians(ang))
                    new_y = x * math.sin(math.radians(ang)) + y * math.cos(math.radians(ang))
                    new_x += cx
                    new_y += cy
                    if c == 0:
                        ori = abs((ori + ang) % 360)
                        c = 1

                elif width - 110 <= mx <= width - 90 and height - 520 <= my <= height - 480:  # right
                    x -= cx
                    y -= cy
                    new_x = x * math.cos(math.radians(-ang)) - y * math.sin(math.radians(-ang))
                    new_y = x * math.sin(math.radians(-ang)) + y * math.cos(math.radians(-ang))
                    new_x += cx
                    new_y += cy
                    if c == 0:
                        ori = abs((ori - ang) % 360)
                        c = 1

                elif width - 150 <= mx < width - 110 and height - 470 <= my <= height - 450:  # bottom
                    new_y = y + speed * (math.cos(math.radians(ori)))
                    new_x = x - speed * (math.sin(math.radians(ori)))
                else:
                    new_x, new_y = x, y

                if new_x != max(600+radius, min(new_x, width - radius)) or new_y != max(radius, min(new_y, height - radius)):
                    flag = False
                    ori=ori2
                    break
                p2.append((int(new_x), int(new_y)))
                


            if flag:
                points = p2.copy()
                ori2=ori
                

    # Ensure the sphere stays within the screen boundaries
    # x = max(radius, min(x, width - radius))
    # y = max(radius, min(y, height - radius))
    # x_l, y_l = max(radius, min(x_l, width - radius)), max(radius, min(y_l, height  - radius))
    # x_r, y_r = max(radius, min(x_r, width - radius)), max(radius, min(y_r, height  - radius))

    # Clear the screen
 
    pygame.draw.rect(screen, (0,0,255), (0, 0, width // 2, height))
    pygame.draw.rect(screen, (0,0,0), (width // 2, 0, width // 2, height))
    diameters = []
    angles = []
    # Draw the red sphere
    for point in points:
        x, y = point
        r = int(1+y/30)
        angle = round((start + ((x-600) * fov*2) / 600) % 360, 1)
        diameter = 2 * r
        angles.append(angle)
        diameters.append(diameter)
        font = pygame.font.Font(None, 20)
        pygame.draw.circle(screen, (138, 18, 26), (int(x), int(y)), r)
        if points[1] == point:
            screen.blit(font.render(f'{2 * r}', True, (83, 235, 46)), (x - 25, y - 20))
            screen.blit(font.render(f'{round(angle, 1)}', True, (223, 239, 119)), (x, y - 20))
        else:
            screen.blit(font.render(f'{2 * r}', True, (83, 235, 46)), (x - 40, y - 20))
            screen.blit(font.render(f'{round(angle, 1)}', True, (223, 239, 119)), (x-15, y - 20))

    # Draw arrows
    pygame.draw.polygon(screen, (0, 0, 255), [(width - 150, height - 530), (width - 110, height - 530),
                                               (width - 130, height - 550)])
    pygame.draw.polygon(screen, (0, 0, 255), [(width - 150, height - 520), (width - 150, height - 480),
                                               (width - 170, height - 500)])
    pygame.draw.polygon(screen, (0, 0, 255), [(width - 110, height - 520), (width - 110, height - 480),
                                               (width - 90, height - 500)])
    pygame.draw.polygon(screen, (0, 0, 255), [(width - 150, height - 470), (width - 110, height - 470),
                                               (width - 130, height - 450)])

    # Display the diameter text
    diameter_text = font.render(f'System Input: Camera vision at night', True, (255, 255, 255))
    screen.blit(diameter_text, (610, 10))
    diameter_text = font.render(f'Diameter in pixels', True, (83, 235, 46))
    screen.blit(diameter_text, (610, 30))
    diameter_text = font.render(f'Angle in degrees', True, (223, 239, 119))
    screen.blit(diameter_text, (610, 50))
    
    ori1=ori+90
    k= 1500*0.8/(2*math.tan(math.radians(fov)))
    distances= [k/(x) for x in diameters]
    area= size(distances, angles)
    if len(prev)>0: 
        if prev[0]!=distances[0]:
            speed1=(distances[0]-prev[0])*frame_rate
    prev=distances.copy()
    distance=min(distances)#round(sum(distances)/3,2)
    if len(positions)<2:
        position=(500, 300)
    else:
        position=positions[-1]
    # d= distances[0]
    #a=math.radians(sum(angles)/3)
    #position= 0.5*pygame.math.Vector2(int(340+cargo_image_rect.centerx+d*math.sin(a)),int(340+cargo_image_rect.centery-d*math.cos(a)))
    
    boat_pos =pygame.math.Vector2(300,150)+0.2*((pygame.math.Vector2(600-position[0], abs(600-position[1]))-pygame.math.Vector2(positions[-2])).rotate(-90)+pygame.math.Vector2(positions[-2])) if len(positions)>=2 else (position[0], abs(position[1]))
    boat_collision_rect = pygame.Rect(boat_pos[0], boat_pos[1], scaled_boat_width, scaled_boat_height)
    boat_image_rect = pygame.Rect(boat_pos[0], boat_pos[1], scaled_boat_width, scaled_boat_height)
    ###
    rotated_boat = pygame.transform.rotate(pygame.transform.scale(boat_images[0], (scaled_boat_width, scaled_boat_height)), -ori1)

    # Update rectangles for collision detection
    boat_collision_rect.update(boat_pos[0], boat_pos[1], rotated_boat.get_width(), rotated_boat.get_height())
    # Update rectangles for image drawing
    
    boat_image_rect.update(boat_pos[0], boat_pos[1], rotated_boat.get_width(), rotated_boat.get_height())

    # Check for collision between rectangles
    if cargo_collision_rect.colliderect(boat_collision_rect):
        collision = True
    else:
        collision = False
        
    pygame.draw.polygon(screen, (65,89,255), [(cargo_image_rect.centerx,cargo_image_rect.centery),(600,int(cargo_image_rect.centery-((600-cargo_image_rect.centerx)*math.tan(math.radians(30))))), (600,600),(600,0),(cargo_image_rect.centerx, 0)])
     # Draw vessels (images)
    rotated_cargo = pygame.transform.rotate(pygame.transform.scale(cargo_image, (scaled_cargo_width, scaled_cargo_height)), 0)
    screen.blit(rotated_cargo, cargo_pos)
    screen.blit(rotated_boat, boat_pos)

    # Draw rectangles around vessels (for image drawing)
    pygame.draw.rect(screen, red if collision else green, cargo_image_rect, 2)
    pygame.draw.rect(screen, red if collision else green, boat_image_rect, 2)

    #pygame.draw.line(screen, [12,28,56], (cargo_image_rect.centerx,cargo_image_rect.centery), (cargo_image_rect.centerx,0), width=1)
    #pygame.draw.line(screen, [12,28,56], (cargo_image_rect.centerx,cargo_image_rect.centery), (int(cargo_image_rect.centerx+((600-cargo_image_rect.centery)*math.tan(math.radians(10)))),600), width=1)
    
    diameter_text = font.render(f'System Output', True, (255, 255, 255))
    screen.blit(diameter_text, (10, 10))
    diameter_text = font.render(f'HFOV: {start}-{start+fov*2}deg', True, (255, 255, 255))
    screen.blit(diameter_text, (310, 10))
    diameter_text = font.render(f'Horizontal resolution:1500px', True, (255, 255, 255))
    screen.blit(diameter_text, (310, 30))
    diameter_text = font.render(f'Actual diameter: 80cm', True, (255, 255, 255))
    screen.blit(diameter_text, (10, 50))
    diameter_text = font.render(f'Activates after two inputs', True, (255, 255, 255))
    screen.blit(diameter_text, (10, 30))
    if start_time!=ptime and start_time!=0:
        etime=time.time()
        e=round(etime-start_time,3)
        start_time=0
    diameter_text = font.render(f'Time complexity per step:{e}s', True, (255, 255, 255))
    screen.blit(diameter_text, (10, 70))
    
    ###
    screen.blit(font.render(f'Type:Boat, Area:{area}sqm', True, (0,0,0)), (boat_pos[0], boat_pos[1]-60))
    screen.blit(font.render(f'Distance:{round(distance,1)}m', True, (0,0,0)),(boat_pos[0], boat_pos[1]-40))
    screen.blit(font.render(f'Velocity:{round(speed1,1)}m/s', True, (0,0,0)),(boat_pos[0], boat_pos[1]-20))
    
    
    # Update the display
    pygame.display.flip()

pygame.quit()
sys.exit()


SystemExit: 

In [1]:
x,y=[1,2],[1,2]
print(x==y)

True
